In [3]:
!pip install crewai crewai[tools] google-generativeai python-dotenv
!pip install langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.3/701.3 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 11.5 MB/s 

#Import Libraries & Access Key:

In [1]:
import os
from google.colab import userdata

from crewai import Agent, Task, Crew, Process, LLM

# --- Setup: Get API Keys ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    BROWSERLESS_API_KEY = userdata.get('BROWSERLESS_API_KEY')
except userdata.SecretNotFoundError as e:
    raise ValueError(f"Error loading secret: {e}. Please ensure both keys are set in Colab secrets.")

if not GOOGLE_API_KEY or not BROWSERLESS_API_KEY:
    raise ValueError("One or both API keys are missing.")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["BROWSERLESS_API_KEY"] = BROWSERLESS_API_KEY

# --- Agent and Crew Initialization ---
# FIX: Use the correct model name format for litellm.
gemini_llm = LLM(
    model="gemini/gemini-2.5-flash", # Correct format for gemini provider
    api_key=os.environ["GOOGLE_API_KEY"]
)

##Agent Building Process##

1- Define a Agents:

In [2]:
# Initialize website scraping tool
# Removing PlaywrightScrapeTool due to asyncio error. Will explore alternatives.

researcher = Agent(
    role='Senior Job Researcher',
    goal="Scour a company's career page to find all available software engineering job listings",
    backstory="""You are an expert market researcher. Your specialty is finding and identifying relevant job opportunities from complex, dynamically-loaded career websites.
    You are a master at sifting through HTML to find the core job data.""",
    verbose=True,
    allow_delegation=False,
    tools=[], # Removed scrape_tool
    llm=gemini_llm  # Pass LLM to the agent directly
)

analyst = Agent(
    role='Job Data Analyst',
    goal='Extract key details from the provided job listings HTML content',
    backstory="""You are a meticulous data analyst. You can read raw HTML and extract specific information with perfect accuracy. You focus on job titles, locations, and application links, formatting them neatly.""",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm # Pass LLM to the agent directly
)

2- Design the Task:

In [3]:
# --- Define Tasks ---
target_urls = [
    "https://www.google.com/about/careers/applications/jobs/results/",
    "https://www.metacareers.com/jobs/",
    "https://hiring.amazon.com/", # Using hiring.amazon.com for job listings
    "https://careers.microsoft.com/", # Added Microsoft
    "https://digitalcareers.infosys.com/", # Added Infosys
    "https://www.accenture.com/us-en/careers/jobsearch", # Added Accenture
    "https://www.tcs.com/careers", # Added TCS
    "https://careers.oracle.com/en/sites/jobsearch/jobs?location=United%20States&locationId=300000000149325", # Added Oracle
    "https://jobs.fidelity.com/en/jobs/?search=&origin=global&lat=&lng=", # Added Fidelity
]
company_names = ["Google", "Meta", "Amazon", "Microsoft", "Infosys", "Accenture", "TCS", "Oracle", "Fidelity", "AutoZone"] # Added company names

research_task = Task(
    description=f'Visit the career pages for {", ".join(company_names)} at the URLs {", ".join(target_urls)} and scrape all job content.',
    expected_output='The full, raw HTML content of the job listings pages from all specified URLs.',
    agent=researcher
)

analysis_task = Task(
    description="""From the provided HTML content from all scraped career pages, identify and extract the following for each
    available job listing: Job Title, Location, and a direct URL link to the application page.
    Format the final output as a clean list of jobs, clearly indicating which company each job belongs to.""",
    expected_output='A formatted list of all jobs from all specified companies with their title, location, and application link.',
    agent=analyst
)

3- Assemble and Run the Crew:

In [4]:
# --- Create and Run the Crew ---
crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, analysis_task],
    verbose=True,
    process=Process.sequential
)

print("Starting the Job Aggregation Crew...")
result = crew.kickoff()
print("\n\n##################################")
print("## Final Report ##")
print(result)

Output hidden; open in https://colab.research.google.com to view.